# Semester 1 Project Submission

Please fill out:
* Student name: Cristian González Ramírez
* Instructor name: Nick McCarty & Julian Ward

In [59]:
# Import all libraries the project will requiere

#For retreiving data
import pandas as pd
import sqlite3
import zipfile

#For analyzing data
from scipy import stats

#For plotting and visulaizations
from matplotlib import pyplot as plt
import seaborn as sns
%matplotlib inline

In [60]:
#Retrieving data to work with
movie_gross_data = pd.read_csv("zippedData/bom.movie_gross.csv.gz")
movies           = pd.read_csv("zippedData/tmdb.movies.csv.gz")
movie_budgets    = pd.read_csv("zippedData/tn.movie_budgets.csv.gz")

movie_info_data  = pd.read_csv("zippedData/rt.movie_info.tsv.gz", sep="\t")

In [61]:
#Oppening SQLite3 zip
with zipfile.ZipFile("zippedData/im.db.zip", "r") as zObject:
        zObject.extractall()

conn = sqlite3.connect("im.db")
cur = conn.cursor()

In [62]:
movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3387 entries, 0 to 3386
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3387 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3359 non-null   float64
 3   foreign_gross   2037 non-null   object 
 4   year            3387 non-null   int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 132.4+ KB


In [63]:
#Cleaning movie_gross_data
movie_gross_data.domestic_gross.replace(",", "", regex=True, inplace=True)
movie_gross_data.foreign_gross.replace(",","",regex=True, inplace=True)

In [64]:
movie_gross_data.foreign_gross.fillna(0, inplace=True)
movie_gross_data.domestic_gross.fillna(0, inplace=True)

In [65]:
movie_gross_data.dropna(inplace = True)

In [66]:
movie_gross_data["total_gross"] = movie_gross_data.foreign_gross.astype(float) +\
                                  movie_gross_data.domestic_gross.astype(float)

In [67]:
movie_gross_data["foreign_gross"] = movie_gross_data.foreign_gross.astype(float)

movie_gross_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3382 entries, 0 to 3386
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           3382 non-null   object 
 1   studio          3382 non-null   object 
 2   domestic_gross  3382 non-null   float64
 3   foreign_gross   3382 non-null   float64
 4   year            3382 non-null   int64  
 5   total_gross     3382 non-null   float64
dtypes: float64(3), int64(1), object(2)
memory usage: 185.0+ KB


In [68]:
movie_gross_data.head()

,title,studio,domestic_gross,foreign_gross,year,total_gross
0,Toy Story 3,BV,415000000.0,652000000.0,2010,1.067000e+09
1,Alice in Wonderland (2010),BV,334200000.0,691300000.0,2010,1.025500e+09
2,Harry Potter and the Deathly Hallows Part 1,WB,296000000.0,664300000.0,2010,9.603000e+08
3,Inception,WB,292600000.0,535700000.0,2010,8.283000e+08
4,Shrek Forever After,P/DW,238700000.0,513900000.0,2010,7.526000e+08


## In the following cells I am trying to give an answer to the following question:

How are the production budgets for movies of different genres in comparison to their income? Would it be profitable?

There are a lot of values in movie_gross_data in which we are not interested, must of them being the $0 USD revenue. Those values may result as strong outliers for future analysis, so it will be convenient to manage our data without them.

In [69]:
movie_general_gross = movie_gross_data[(movie_gross_data.foreign_gross != 0) &
                                       (movie_gross_data.domestic_gross !=0)
                                      ].sort_values(by=["total_gross"], ascending=False)
movie_general_gross

,title,studio,domestic_gross,foreign_gross,year,total_gross
727,Marvel's The Avengers,BV,623400000.0,895500000.0,2012,1.518900e+09
1875,Avengers: Age of Ultron,BV,459000000.0,946400000.0,2015,1.405400e+09
3080,Black Panther,BV,700100000.0,646900000.0,2018,1.347000e+09
328,Harry Potter and the Deathly Hallows Part 2,WB,381000000.0,960500000.0,2011,1.341500e+09
2758,Star Wars: The Last Jedi,BV,620200000.0,712400000.0,2017,1.332600e+09
...,...,...,...,...,...,...
711,I'm Glad My Mother is Alive,Strand,8700.0,13200.0,2011,2.190000e+04
322,The Thorn in the Heart,Osci.,7400.0,10500.0,2010,1.790000e+04
1110,Cirkus Columbia,Strand,3500.0,9500.0,2012,1.300000e+04
715,Aurora,CGld,5700.0,5100.0,2011,1.080000e+04


In [70]:
query = '''
    SELECT
        movie_id AS "ID",
        primary_title AS "Title",
        original_title AS "Original Title",
        genres AS "Genres",
        averagerating AS "Rating"
    FROM movie_basics
        JOIN movie_ratings
        USING (movie_id)
'''

pd.read_sql(query,conn)

,ID,Title,Original Title,Genres,Rating
0,tt0063540,Sunghursh,Sunghursh,"Action,Crime,Drama",7.0
1,tt0066787,One Day Before the Rainy Season,Ashad Ka Ek Din,"Biography,Drama",7.2
2,tt0069049,The Other Side of the Wind,The Other Side of the Wind,Drama,6.9
3,tt0069204,Sabse Bada Sukh,Sabse Bada Sukh,"Comedy,Drama",6.1
4,tt0100275,The Wandering Soap Opera,La Telenovela Errante,"Comedy,Drama,Fantasy",6.5
...,...,...,...,...,...
73851,tt9913084,Diabolik sono io,Diabolik sono io,Documentary,6.2
73852,tt9914286,Sokagin Çocuklari,Sokagin Çocuklari,"Drama,Family",8.7
73853,tt9914642,Albatross,Albatross,Documentary,8.5
73854,tt9914942,La vida sense la Sara Amat,La vida sense la Sara Amat,None,6.6
